In [1]:
#secure this key
cohere_api_key = "xjxmQmL32LULRG1jJWp4loHT5i7yMBtQBUx0da8E"   #jjQan7pmPZAnZFDZCddyDxYCPieyaWI0rG1N0gky 
                                                              #v5OEWWEgBMIyH5Fdqy7Y58x722csvT0cDYeYkYDM
                                                              

# 1. Resume Screener using GenAI

### Setup Cohere API

In [2]:
import cohere

# Initialize the Cohere client with your API key
co = cohere.Client(cohere_api_key)

### Creating Synthetic Data

In [3]:
import random
import pandas as pd

# Define sample resumes and job descriptions
resume_templates = [
    "My name is {name}. I have {years} years of experience in {skill1} and {skill2}. I hold a {degree} in {major} from {university}.",
    "I am {name}, with {years} years of experience in {skill1} and {skill2}. I hold a {degree} in {major}.",
    "{name} has worked with {skill1} and {skill2} for {years} years. Completed {degree} in {major} at {university}."
]

job_templates = [
    "We are looking for a candidate with expertise in {skill1} and {skill2}, holding a {degree} in {major}.",
    "Looking for a professional with {years} years of experience in {skill1} and {skill2}. Must have a {degree} in {major}.",
]

# Define data points
names = ["Alice", "Bob", "Charlie", "David", "Eva"]
skills = ["Python", "Machine Learning", "Data Science", "Java", "Cloud"]
degrees = ["Bachelor's", "Master's", "PhD"]
majors = ["Computer Science", "Data Science", "Software Engineering"]
universities = ["MIT", "Stanford", "Harvard", "Berkeley"]

# Function to create random resumes
def create_resume():
    name = random.choice(names)
    years = random.randint(1, 10)
    skill1, skill2 = random.sample(skills, 2)
    degree = random.choice(degrees)
    major = random.choice(majors)
    university = random.choice(universities)
    
    resume = random.choice(resume_templates).format(name=name, years=years, skill1=skill1, skill2=skill2, degree=degree, major=major, university=university)
    return resume

# Function to create random job descriptions
def create_job_description():
    years = random.randint(1, 10)
    skill1, skill2 = random.sample(skills, 2)
    degree = random.choice(degrees)
    major = random.choice(majors)
    
    job_description = random.choice(job_templates).format(years=years, skill1=skill1, skill2=skill2, degree=degree, major=major)
    return job_description

# Generate 1000 synthetic resumes and job descriptions
num_samples = 1000
resumes = [create_resume() for _ in range(num_samples)]
job_descriptions = [create_job_description() for _ in range(num_samples)]

# Create a DataFrame
df = pd.DataFrame({
    'resume': resumes,
    'job_description': job_descriptions
})

df.head()

,resume,job_description
0,"I am Alice, with 4 years of experience in Pyth...",Looking for a professional with 7 years of exp...
1,"I am Charlie, with 10 years of experience in P...",Looking for a professional with 10 years of ex...
2,"I am Charlie, with 8 years of experience in Py...",Looking for a professional with 7 years of exp...
3,My name is Bob. I have 8 years of experience i...,Looking for a professional with 9 years of exp...
4,My name is Bob. I have 7 years of experience i...,We are looking for a candidate with expertise ...


### Get Text Embeddings using Cohere

In [4]:
# Get embeddings for resumes
resume_embeddings = co.embed(texts=df['resume'].tolist(), model='embed-english-v2.0').embeddings

# Get embeddings for job descriptions
job_embeddings = co.embed(texts=df['job_description'].tolist(), model='embed-english-v2.0').embeddings

# Convert to DataFrame for easier manipulation
resume_embeddings_df = pd.DataFrame(resume_embeddings)
job_embeddings_df = pd.DataFrame(job_embeddings)


### Measure Similarity between CV & JD

In [5]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to compute cosine similarity between two vectors
def compute_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])[0][0]

# Compute similarity for each resume and job description pair
df['similarity_score'] = [
    compute_similarity(resume_embeddings[i], job_embeddings[i]) for i in range(num_samples)
]

# Display the results
df[['resume', 'job_description', 'similarity_score']].head()


,resume,job_description,similarity_score
0,"I am Alice, with 4 years of experience in Pyth...",Looking for a professional with 7 years of exp...,0.681102
1,"I am Charlie, with 10 years of experience in P...",Looking for a professional with 10 years of ex...,0.460076
2,"I am Charlie, with 8 years of experience in Py...",Looking for a professional with 7 years of exp...,0.564380
3,My name is Bob. I have 8 years of experience i...,Looking for a professional with 9 years of exp...,0.635134
4,My name is Bob. I have 7 years of experience i...,We are looking for a candidate with expertise ...,0.533779


### Screen resume based on similarity scores

In [6]:
# Set threshold for suitable resumes
threshold = 0.7

# Filter resumes based on the threshold
df['is_suitable'] = df['similarity_score'] > threshold

# Show suitable resumes
suitable_resumes = df[df['is_suitable']]
suitable_resumes[['resume', 'job_description', 'similarity_score']].head()


,resume,job_description,similarity_score
11,My name is David. I have 1 years of experience...,We are looking for a candidate with expertise ...,0.706175
16,My name is Eva. I have 2 years of experience i...,We are looking for a candidate with expertise ...,0.706042
19,"I am Alice, with 2 years of experience in Pyth...",Looking for a professional with 8 years of exp...,0.779964
27,My name is Eva. I have 2 years of experience i...,Looking for a professional with 3 years of exp...,0.701383
28,"I am Bob, with 9 years of experience in Java a...",Looking for a professional with 4 years of exp...,0.711180


# 2. Cohere Summarization Module

### Extractive Summarization

In [7]:
import cohere
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Initialize Cohere client
co = cohere.Client(cohere_api_key)

def extractive_summarization(text, num_sentences=3):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    
    # Get embeddings for each sentence
    sentence_embeddings = co.embed(texts=sentences).embeddings
    
    # Get the embedding for the entire text (summary vector)
    text_embedding = co.embed(texts=[text]).embeddings[0]
    
    # Calculate cosine similarity between the text embedding and sentence embeddings
    similarities = cosine_similarity([text_embedding], sentence_embeddings)[0]
    
    # Rank sentences by similarity
    ranked_sentences = [sentences[i] for i in np.argsort(similarities)[::-1]]
    
    # Select the top N sentences
    summary = ' '.join(ranked_sentences[:num_sentences])
    
    return summary

# Test extractive summarization
text = '''Generative AI, also referred to as GenAI, allows users to input a variety of
    prompts to generate new content, such as text, images, videos, sounds, code, 
    3D designs, and other media. It “learns” and is trained on documents and artifacts that already exist online.'''



print("Extractive Summary:", extractive_summarization(text))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abiju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
default model on embed will be deprecated in the future, please specify a model in the request.
default model on embed will be deprecated in the future, please specify a model in the request.


Extractive Summary: Generative AI, also referred to as GenAI, allows users to input a variety of
    prompts to generate new content, such as text, images, videos, sounds, code, 
    3D designs, and other media. It “learns” and is trained on documents and artifacts that already exist online.


### Abstractive Summarization

In [8]:
def abstractive_summarization(text):
    response = co.generate(
        model='command-xlarge-nightly',  # Use a Cohere large model for better performance
        prompt=f"Summarize the following text:\n\n{text}\n\nSummary:",
        max_tokens=100
    )
    
    summary = response.generations[0].text.strip()
    return summary

# Test abstractive summarization
abstractive_summary = abstractive_summarization(text)
print("Abstractive Summary:", abstractive_summary)


unknown field: parameter model is not a valid field


Abstractive Summary: Generative AI is a technology that enables users to create new content, including text, images, videos, sounds, code, and 3D designs, by inputting prompts. It is trained by learning from existing online documents and artifacts.


### Hybrid Summarization

In [9]:
def hybrid_summarization(text, num_sentences=3):
    # First perform extractive summarization
    extracted_summary = extractive_summarization(text, num_sentences)
    
    # Then perform abstractive summarization on the extracted sentences
    abstractive_summary = abstractive_summarization(extracted_summary)
    
    return abstractive_summary

# Test hybrid summarization
hybrid_summary = hybrid_summarization(text)
print("Hybrid Summary:", hybrid_summary)

default model on embed will be deprecated in the future, please specify a model in the request.
default model on embed will be deprecated in the future, please specify a model in the request.
unknown field: parameter model is not a valid field


Hybrid Summary: Generative AI is a technology that can create various types of content, including text, images, videos, sounds, code, and 3D designs, based on user-inputted prompts. It is trained by learning from existing online documents and artifacts.


# 3.Generic Recommendation Engine

### Generate Synthetic data

In [10]:
import pandas as pd
import random

# Lists of synthetic data
users = ['Jane', 'Jack', 'Emily', 'Sarah', 'John', 'Michael', 'Anna', 'David', 'Lucy', 'James']
cosmetics = ['Lipstick', 'Eyeliner', 'Kajal', 'Foundation', 'Blush', 'Mascara', 'Nail Polish', 'Concealer', 'Highlighter', 'Eyeshadow']
brands = ['BrandA', 'BrandB', 'BrandC', 'BrandD', 'BrandE']
categories = ['Lip', 'Eye', 'Face', 'Nail']
ratings = [1, 2, 3, 4, 5]

# Generate synthetic cosmetic product data
data = {
    'user': [random.choice(users) for _ in range(1000)],
    'product_name': [random.choice(cosmetics) for _ in range(1000)],
    'brand': [random.choice(brands) for _ in range(1000)],
    'category': [random.choice(categories) for _ in range(1000)],
    'rating': [random.choice(ratings) for _ in range(1000)]
}

df = pd.DataFrame(data)
df.head()

,user,product_name,brand,category,rating
0,David,Nail Polish,BrandC,Lip,3
1,James,Foundation,BrandA,Face,3
2,Emily,Lipstick,BrandB,Eye,2
3,James,Eyeshadow,BrandD,Eye,1
4,Michael,Blush,BrandE,Lip,2


### Get embeddings using Cohere

In [11]:
import cohere
from sklearn.metrics.pairwise import cosine_similarity

# Initialize Cohere client (replace 'your-api-key' with your actual API key)
co = cohere.Client(cohere_api_key)

# Generate synthetic product descriptions (for demonstration purposes)
df['product_description'] = df['product_name'] + ' by ' + df['brand'] + ', category: ' + df['category']

# Get product embeddings using Cohere
product_descriptions = df['product_description'].tolist()
embeddings = co.embed(texts=product_descriptions, model='embed-english-v2.0').embeddings


### Build Recommendation Function

In [12]:
def recommend_products(user, df, embeddings, top_n=5):
    # Get the user's previously rated products
    user_data = df[df['user'] == user]
    
    if user_data.empty:
        return f"No recommendations available for {user}."
    
    # Get the most recent product that the user interacted with
    recent_product = user_data.iloc[-1]['product_description']
    
    # Embed the recent product description
    recent_product_embedding = co.embed(texts=[recent_product], model='embed-english-v2.0').embeddings[0]
    
    # Calculate cosine similarity between the user's recent product and all other products
    similarities = cosine_similarity([recent_product_embedding], embeddings)[0]
    
    # Get top N most similar products, excluding the one the user already interacted with
    df['similarity'] = similarities
    recommendations = df[~df['product_description'].isin(user_data['product_description'])].sort_values(by='similarity', ascending=False).head(top_n)
    
    return recommendations[['product_name', 'brand', 'category', 'similarity']]

# Example recommendation for user 'Jane'
recommendations = recommend_products('Jane', df, embeddings, top_n=5)
print(recommendations)


    product_name   brand category  similarity
134   Foundation  BrandB     Nail    0.990148
629   Foundation  BrandA     Nail    0.989722
961   Foundation  BrandA     Nail    0.989722
469   Foundation  BrandD     Nail    0.989003
354   Foundation  BrandD     Nail    0.989003
